In [ ]:
import pandas as pd
import numpy as np
import jalali
import statistics
import plotly.graph_objects as go

# Modify Date And Water Table
# Date Type: 
#   1.Gregorian (Text): 2000-11-1
#   2.Persian (Text): 1399-1-1
# Date & Value:
#   Column of Pandas Dataframe
def convert_to_day_15(info, date, value, date_type="persian"):
    df = info.copy()
    if date_type == "gregorian":
        df["DATE_GREGORIAN"] = date.apply(pd.to_datetime)
        df["DATE_PERSIAN"] = list(
            map(
                lambda i: jalali.Gregorian(i.date()).persian_string(),
                df["DATE_GREGORIAN"]
            )
        )
    elif date_type == "persian":
        df["DATE_PERSIAN"] = date
        df["DATE_GREGORIAN"] = list(
            map(
                lambda i: jalali.Persian(i).gregorian_string(),
                df["DATE_PERSIAN"]
            )
        )
        df["DATE_GREGORIAN"] = df["DATE_GREGORIAN"].apply(pd.to_datetime)    
    else:
        pass
    df["VALUE"] = value        
    df["DELTA_DAY"] = df["DATE_GREGORIAN"].diff().dt.days
    df["DATE_PERSIAN_NEW"] = list(
        map(
            lambda i: f"{int(i.split('-')[0])}-{int(i.split('-')[1])}-{15}",
            df["DATE_PERSIAN"]
        )
    )
    df["DATE_GREGORIAN_NEW"] = list(
        map(
            lambda i: jalali.Persian(i).gregorian_string(),
            df["DATE_PERSIAN_NEW"]
        )
    )
    df["DATE_GREGORIAN_NEW"] = df["DATE_GREGORIAN_NEW"].apply(pd.to_datetime)
    df["VALUE_NEW"] = df["VALUE"]
    A = []
    A.append(df["VALUE"][0])
    for i in range(1, len(df)):
        if int(df["DATE_PERSIAN"][i].split('-')[2]) >= 15:
            NEW_VALUE = df["VALUE"][i-1] + ((((df["DATE_GREGORIAN_NEW"][i] - df["DATE_GREGORIAN"][i-1]).days) / ((df["DATE_GREGORIAN"][i] - df["DATE_GREGORIAN"][i-1]).days)) * (df["VALUE"][i] - df["VALUE"][i-1]))
            A.append(NEW_VALUE)
        else:
            NEW_VALUE = df["VALUE"][i] + ((((df["DATE_GREGORIAN_NEW"][i] - df["DATE_GREGORIAN"][i]).days) / ((df["DATE_GREGORIAN"][i+1] - df["DATE_GREGORIAN"][i]).days)) * (df["VALUE"][i+1] - df["VALUE"][i]))
            A.append(NEW_VALUE)
    df["VALUE_NEW"] = A
    return df


def create_date_day15(min, max):    
    result = []
    min_list = list(map(lambda x: int(x), min.split("-")))
    max_list = list(map(lambda x: int(x), max.split("-")))
    for y in range(min_list[0], max_list[0] + 1):
        for m in range(1, 13):
            result.append(f"{y}-{m}-15")

    result = pd.DataFrame(
        {"DATE_PERSIAN" : result}
    )
    result['DATE_GREGORIAN'] = result.apply(
        lambda x: jalali.Persian(x["DATE_PERSIAN"]).gregorian_string(), 
        axis=1
    )
    result["DATE_GREGORIAN"] = result["DATE_GREGORIAN"].apply(pd.to_datetime)
    result = result[result["DATE_GREGORIAN"] >= pd.to_datetime(jalali.Persian(min).gregorian_string())]
    result = result[result["DATE_GREGORIAN"] <= pd.to_datetime(jalali.Persian(max).gregorian_string())]
    result["DATE_GREGORIAN"] = result["DATE_GREGORIAN"].apply(pd.to_datetime)  
    return result


# Read Data And GeoInfo
xls = pd.ExcelFile('Data/CSV/HydrographData.xlsx')
Data = pd.read_excel(xls, sheet_name='Data')
GeoInfo = pd.read_excel(xls, sheet_name='GeoInfo')

# GeoInfo
COLs = ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'LOCATION_NAME']
GeoInfo[COLs] = GeoInfo[COLs].apply(lambda x: x.str.rstrip())
GeoInfo[COLs] = GeoInfo[COLs].apply(lambda x: x.str.lstrip())

# Data
COLs = ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'LOCATION_NAME']
Data[COLs] = Data[COLs].apply(lambda x: x.str.rstrip())
Data[COLs] = Data[COLs].apply(lambda x: x.str.lstrip())

# Convert Date
Data["DATE_GREGORIAN_RAW"] = Data["DATE_GREGORIAN_RAW"].apply(pd.to_datetime)

Data['DATE_CHECK'] = np.where(
    Data["DATE_PERSIAN_RAW"].isna(),
    np.where(
        Data["DATE_GREGORIAN_RAW"].isna(),
        np.NaN,
        "G"
    ),
    "P"  
)

Data['DATE_PERSIAN_RAW'] = Data.apply(
    lambda x: jalali.Gregorian(x["DATE_GREGORIAN_RAW"].date()).persian_string() if x["DATE_CHECK"] == "G" else x["DATE_PERSIAN_RAW"], 
    axis=1
)

Data['DATE_GREGORIAN_RAW'] = Data.apply(
    lambda x: jalali.Persian(x["DATE_PERSIAN_RAW"]).gregorian_string() if x["DATE_CHECK"] == "P" else x["DATE_GREGORIAN_RAW"], 
    axis=1
)

Data["DATE_GREGORIAN_RAW"] = Data["DATE_GREGORIAN_RAW"].apply(pd.to_datetime)


Data.drop(['DATE_CHECK'], axis=1, inplace=True)

Data.sort_values(
    by=["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", "DATE_GREGORIAN_RAW"], 
    inplace=True
)

# Remove NanN Data In Column "WATER_TABLE_RAW"
Data.dropna(
    subset=["WATER_TABLE_RAW"],
    inplace=True
)

Data.reset_index(
    inplace=True,
    drop=True
)

wt_date_converted = convert_to_day_15(
    info=Data[["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME"]],
    date=Data["DATE_PERSIAN_RAW"],
    value=Data["WATER_TABLE_RAW"],
    date_type="persian"
)[["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", "DATE_PERSIAN", "DATE_PERSIAN_NEW", "DATE_GREGORIAN", "DATE_GREGORIAN_NEW", "VALUE_NEW"]]

wt_date_converted.columns = ["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", "DATE_PERSIAN_RAW", "DATE_PERSIAN", "DATE_GREGORIAN_RAW","DATE_GREGORIAN", "WATER_TABLE"]

Data = Data.merge(
    right=wt_date_converted,
    how="left",
    on=["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", "DATE_PERSIAN_RAW", "DATE_GREGORIAN_RAW"]
)

tmp = pd.DataFrame()

for mn in list(Data["MAHDOUDE_NAME"].unique()):
    for an in list(Data["AQUIFER_NAME"].unique()):
        for ln in list(Data["LOCATION_NAME"].unique()):
            df = Data[(Data["MAHDOUDE_NAME"] == mn) & (Data["MAHDOUDE_NAME"] == an) & (Data["LOCATION_NAME"] == ln)]
            date = create_date_day15(
                min = df.DATE_PERSIAN.min(),
                max = df.DATE_PERSIAN.max()
            )
            df = date.merge(
                df,
                how="left",
                on=["DATE_PERSIAN", "DATE_GREGORIAN"]
            )
            df["WATER_TABLE_LINEAR"] = df["WATER_TABLE"].interpolate(method='linear')
            df["WATER_TABLE_LINEAR"] = df["WATER_TABLE_LINEAR"].interpolate(method='ffill')
            df["WATER_TABLE_LINEAR"] = df["WATER_TABLE_LINEAR"].interpolate(method='bfill')
            df["WATER_TABLE_SPLINE"] = df["WATER_TABLE"].interpolate(method='akima')
            df["WATER_TABLE_SPLINE"] = df["WATER_TABLE_SPLINE"].interpolate(method='ffill')
            df["WATER_TABLE_SPLINE"] = df["WATER_TABLE_SPLINE"].interpolate(method='bfill')
            df["MAHDOUDE_NAME"] = mn
            df["MAHDOUDE_CODE"] = int(df["MAHDOUDE_CODE"].unique()[0])
            df["AQUIFER_NAME"] = an
            df["LOCATION_NAME"] = ln
            df["DATA_STATE"] = df["DATA_STATE"].fillna("M")
            df["STORAGE_COEFFICIENT_LOCATION"] = df["STORAGE_COEFFICIENT_LOCATION"].unique()[0]

            df = df[[
                "MAHDOUDE_NAME", "MAHDOUDE_CODE", "AQUIFER_NAME", "LOCATION_NAME",
                "DATE_GREGORIAN", "DATE_PERSIAN",
                "WATER_TABLE", "WATER_TABLE_LINEAR", "WATER_TABLE_SPLINE", "STORAGE_COEFFICIENT_LOCATION", "THISSEN_LOCATION", "THISSEN_AQUIFER",
                "DATA_STATE", "NO_MEASURE_CODE", "INFO",
                "DATE_GREGORIAN_RAW", "DATE_PERSIAN_RAW", "WATER_TABLE_RAW"	
            ]]

            tmp = pd.concat([tmp, df], axis=0)

Data = tmp.copy()

Data.to_excel("df.xlsx")


# Data['TMP'] = (Data['STORAGE_COEFFICIENT_LOCATION'] * Data['THISSEN_LOCATION']) / Data['THISSEN_AQUIFER']

# tmp = Data.groupby(by=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN']).sum().reset_index()[
#             ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN', 'TMP']].rename(columns={'TMP': 'STORAGE_COEFFICIENT_AQUIFER'})

# Data = Data.merge(
#     right=tmp,
#     how='left',
#     on=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN'])\
#         .sort_values(["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", 'DATE_GREGORIAN'])\
#             .drop(['TMP'], axis=1)

# del tmp

# SELECTED_LEVEL = 'LEVEL_SRTM'

# Data = Data.merge(
#     right=GeoInfo[['MAHDOUDE_NAME', 'AQUIFER_NAME', 'LOCATION_NAME', SELECTED_LEVEL]],
#     how='left',
#     on=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'LOCATION_NAME']
# ).rename(columns={SELECTED_LEVEL: 'LEVEL'})

# Data['HEAD_LOCATION'] = Data['LEVEL'] - Data['WATER_TABLE']

# Data['TMP'] = (Data['HEAD_LOCATION'] * Data['THISSEN_LOCATION']) / Data['THISSEN_AQUIFER']

# tmp = Data.groupby(by=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN']).sum().reset_index()[
#             ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN', 'TMP']].rename(columns={'TMP': 'HEAD_AQUIFER_THISSEN'})

# Data = Data.merge(
#     right=tmp,
#     how='left',
#     on=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN'])\
#         .sort_values(["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", 'DATE_GREGORIAN'])\
#             .drop(['TMP'], axis=1)

# del tmp

# tmp = Data[['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'HEAD_LOCATION']]\
#     .groupby(by=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN'])\
#         .agg({'HEAD_LOCATION': [statistics.mean, statistics.geometric_mean, statistics.harmonic_mean]})\
#             .reset_index()

# tmp.columns = [col for col in tmp.columns]

# tmp.columns = ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'HEAD_AQUIFER_ARITHMETIC', 'HEAD_AQUIFER_GEOMETRIC', 'HEAD_AQUIFER_HARMONIC']

# Data = Data.merge(
#     right=tmp,
#     how='left',
#     on=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN'])\
#         .sort_values(["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", 'DATE_GREGORIAN'])

# del tmp


Data

In [ ]:
for mn in list(Data["MAHDOUDE_NAME"].unique()):
    for an in list(Data["AQUIFER_NAME"].unique()):
        for ln in list(Data["LOCATION_NAME"].unique()):

            df = Data[(Data["MAHDOUDE_NAME"] == mn) & (Data["MAHDOUDE_NAME"] == an) & (Data["LOCATION_NAME"] == ln)]

            # Create Traces
            fig = go.Figure()

            fig.add_trace(
                go.Scatter(
                    x=df["DATE_GREGORIAN"],
                    y=df["WATER_TABLE"],
                    mode='markers',
                    name='Original',
                    marker=dict(
                        color='black',
                        size=12,
                    )
                )
            )

            fig.add_trace(
                go.Scatter(
                    x=df["DATE_GREGORIAN"],
                    y=df["WATER_TABLE_SPLINE"],
                    mode='lines+markers',
                    name='Spline',
                    line=dict(
                        color='green',
                        width=2
                    ),
                    marker=dict(
                        color='green',
                        size=8,
                        opacity=1,
                    )
                ),
            )

            fig.add_trace(
                go.Scatter(
                    x=df["DATE_GREGORIAN"],
                    y=df["WATER_TABLE_LINEAR"],
                    mode='lines+markers',
                    name='Linear',
                    line=dict(
                        color='red',
                        width=1
                    ),
                    marker=dict(
                        color='red',
                        size=4,
                        opacity=1,
                    )
                )
            )

            fig.update_layout(
                title=df["LOCATION_NAME"].unique()[0],
                yaxis_title="Water Table (m)",
                font=dict(
                    family="Courier New, monospace",
                    size=18,
                    color="RebeccaPurple"
                )
            )

            fig.show()
